In [195]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [196]:
from lib import gradient_descent, gauss_newton
import numpy as np

In [197]:
def f(x: np.ndarray) -> np.ndarray:
    assert x.shape == (1, 2)
    x = x.reshape(2)
    return np.array([x[0] ** 4 + x[1] ** 2 - 2 * x[0]**2]).reshape(1)

def df(x: np.ndarray) -> np.ndarray:
    assert x.shape == (1, 2)
    x = x.reshape(2)
    return np.array([4 * x[0] ** 3 - 4 * x[0], 2 * x[1]]).reshape(1, 2)

# f has local minima at (1, 0) and (-1, 0)

x0 = np.random.randn(1, 2)
alpha = 0.1
max_iter = 1000

In [198]:
gradient_descent(f, df, x0, alpha, max_iter)

array([[-1.00000000e+00, -1.78603715e-97]])

In [199]:
h = df(x0).T @ df(x0)
h

array([[ 2.0250878 , -4.13195344],
       [-4.13195344,  8.43076495]])

In [200]:
np.linalg.inv(h)

LinAlgError: Singular matrix

In [ ]:
gauss_newton(f, df, x0, alpha, max_iter)

0


LinAlgError: Singular matrix